# 正负情绪判别

在cnsenti中，每个词语会被赋予一个情感极性值，取值范围为-1到1，表示词语的负面情感程度和正面情感程度。根据每个词语的情感极性值，以及情感词前是否有强度副词的修饰作用以及是否有否定词的情感语义反转作用，cnsenti会计算出整个文本的情感极性评分。情感极性评分的取值范围为-1到1，越接近1表示越积极，越接近-1表示越消极，趋近于0表示中性。

In [2]:
from cnsenti import Sentiment

使用sentiment_count函数统计得到的结果只是简单地计数，而没有考虑到这些细节。

相比之下，sentiment_calculate函数对每个词语的情感极性值进行了加权，同时考虑到修饰副词和否定词的影响。例如，词语“非常高兴”会被赋予更高的情感极性值，因为它前面有一个强度副词“非常”。而“不开心”则会被赋予更低的情感极性值，因为它前面有一个否定词“不”。

In [4]:
senti = Sentiment()
test_text= '很喜欢罗勒家的番茄🍅锅，果然是爱喝汤星人'
result1 = senti.sentiment_count(test_text)
result2 = senti.sentiment_calculate(test_text)
print('sentiment_count',result1)
print('sentiment_calculate',result2)

sentiment_count {'words': 14, 'sentences': 1, 'pos': 1, 'neg': 0}
sentiment_calculate {'sentences': 1, 'words': 14, 'pos': 39.0, 'neg': 0.0}


In [12]:
from cnsenti import Emotion

emotion = Emotion()
test_text = '很喜欢罗勒家的番茄🍅锅，果然是爱喝汤星人'
result = emotion.emotion_count(test_text)
print(result)

{'words': 15, 'sentences': 1, '好': 1, '乐': 0, '哀': 0, '怒': 0, '惧': 0, '恶': 0, '惊': 0}


# 文本情绪多分类

## 直接使用预训练模型进行预测

### 数据集读取

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/天池比赛情绪分类训练数据集.csv')
data

,index,content,label
0,0,'你知道多伦多附近有什么吗?哈哈有破布耶...真的书上写的你听哦...你家那块破布是世界上最...,sadness
1,1,"平安夜,圣诞节,都过了,我很难过,和妈妈吵了两天,以死相逼才终止战争,现在还处于冷战中。",sadness
2,2,"我只是自私了一点,做自己想做的事情!",sadness
3,3,"让感动的不仅仅是雨过天晴,还有泪水流下来的迷人眼神。",happiness
4,4,好日子,happiness
...,...,...,...
34411,35689,跟两位好友吃了一顿手包饺子九菜味煎的食得好饱[馋嘴]【能吃到饱的感觉真幸福满足,happiness
34412,35690,"你只看到我的旧坑,却没看到我又开了新坑;你有你的规则,我有我的潜规则;你否定bg,我决定必基...",happiness
34413,35691,"欠你的,伤你的,还有机会补偿吗?",sadness
34414,35692,"昨天晚上失眠啊,失了一晚上啊,早上要睡着结果要上班了。今天上午还开一上午的会,来不起了啊。。[泪]",sadness


In [3]:
data['label'].unique()

array(['sadness', 'happiness', 'anger', 'fear', 'disgust'], dtype=object)

### 加载第三方库

In [6]:
import statistics
import numpy as np
import pandas as pd

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm
from sklearn.metrics import classification_report

### 导入transformer预训练模型

In [7]:
transfomer = "nanaaaa/emotion_chinese_english"
model = AutoModelForSequenceClassification.from_pretrained(transfomer,ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(transfomer)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tr

In [8]:
text_list = data['content'].tolist()
label_list = data['label'].tolist()
unique_labels = sorted(list(set(label_list)))

## 直接通过预训练模型进行文本情感分类

### 查看模型的标签

In [9]:
labels = model.config.id2label
print(labels)

{0: 'joy', 1: 'sadness', 2: 'anger', 3: 'fear', 4: 'love'}


In [145]:
predict_result = []
for text in tqdm(text_list[:10]):
    # 对文本进行tokenize和编码
    inputs = tokenizer(text, return_tensors="pt")
    # 使用模型进行预测
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits)
    # 输出预测结果
    predict_result.append(labels[predicted_class.item()])

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.32it/s]


In [146]:
result = pd.DataFrame({'text': text_list[:10],'True': label_list[:10], 'predict': predict_result,})
result

,text,True,predict
0,'你知道多伦多附近有什么吗?哈哈有破布耶...真的书上写的你听哦...你家那块破布是世界上最...,sadness,fear
1,"平安夜,圣诞节,都过了,我很难过,和妈妈吵了两天,以死相逼才终止战争,现在还处于冷战中。",sadness,sadness
2,"我只是自私了一点,做自己想做的事情!",sadness,love
3,"让感动的不仅仅是雨过天晴,还有泪水流下来的迷人眼神。",happiness,joy
4,好日子,happiness,joy
5,每天都学到这么晚。总是兴致勃勃想着明天要七点起床继续看书。就是闹钟从来不知道何时关的。一觉醒...,like,sadness
6,终于把身份证办了。,happiness,joy
7,奖学金终于到账呐......终于可以填翻笔数呐~再唔黎我就穷死噶呐~thankgod![嘻嘻...,happiness,joy
8,"高中的时候,一次上午上课,老师大喊:谁敢不服我!一个同学心血来潮,大喊道:我不服!老师愣了一...",happiness,anger
9,汉子你尽快弃暗投明吧我忙得像只狗没空理你啊,sadness,anger


## 通过prompt使用预训练模型进行预测

#### 通过将premise和prompt两个文本序列输入到指定的预训练模型中，获取模型在每一个情绪标签下的输出得分，通过得分值确定输入文本序列所属的类别。

In [10]:
y_pred = []
y_true = []
probs_emotions = []
for index, text in enumerate(tqdm(text_list)):
    premise = text
    probs = []
    with torch.no_grad():
        for label in unique_labels:
            prompt = label
            x = tokenizer.encode(premise[:512], prompt, return_tensors='pt',truncation_strategy='do_not_truncate')
            x = x.to(device)
            logits = model(x)[0]
            entail_contradiction_logits = logits[:,[0,1]]
            prob_label_is_true = entail_contradiction_logits.softmax(dim=1)[:,1]
            probs.append(prob_label_is_true.detach().cpu().numpy()[0])
    y_pred.append(unique_labels[np.argmax(np.array(probs))])
    probs_emotions.append(probs)
    y_true.append(label_list[index])

100%|██████████████████████████████████████████████████████████████████████████| 34416/34416 [2:17:43<00:00,  4.16it/s]


In [ ]:
result = pd.DataFrame({'text': text_list,'True': y_true, 'predict': y_pred,})
result.head()

In [11]:
from sklearn.metrics import classification_report
# y_test为测试集真实标签， y_pred为测试集预测的标签
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

       anger       0.09      0.15      0.11      4068
     disgust       0.09      0.08      0.09      4347
        fear       0.01      0.03      0.02       590
   happiness       0.12      0.01      0.02     12936
     sadness       0.29      0.51      0.37     12475

    accuracy                           0.21     34416
   macro avg       0.12      0.15      0.12     34416
weighted avg       0.17      0.21      0.16     34416

